# Imports

In [1]:
import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

import tensorflow as tf
print(f'TF version: {tf.__version__}')
import tensorflow_addons as tfa
from tensorflow.keras import layers

import transformers
print(f'transformers version: {transformers.__version__}')
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# import sys
# sys.path.append('../input/iterativestratification')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

TF version: 2.6.4
transformers version: 4.20.1


In [2]:
class CFG:
    wandb = False
    DEBUG = False
    TO_KAGGLE = True
    MEMO = "tensorflow ベースライン"
    file_name = "008"
    score_path = "/home/jupyter/output/scores/scores003.csv"
    model="roberta-base"
    epochs = 10
    trn_fold=[0,1,2,3,4]
    n_fold=5
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    
if CFG.DEBUG:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [3]:
import os
import datetime
import pickle
import glob

# ====================================================
# datetime
# ====================================================
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
date = now.strftime('%Y%m%d')
date2 = now.strftime('%Y%m%d%H%M')


# ====================================================
# file_path
# ====================================================
if "/" in CFG.model:
    model_name = CFG.model.split("/")[1]
else:
    model_name = CFG.model

path ="/home/jupyter/feedback-prize-english-language-learning/"
if CFG.DEBUG:
    OUTPUT_DIR = f'/home/jupyter/output/ex/DEBUG/{model_name}/{CFG.file_name}/{date2}/'
else:
    OUTPUT_DIR = f'/home/jupyter/output/ex/{model_name}/{CFG.file_name}/{date2}/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR+"tokenizer/")
    
# ====================================================
# wandb 
# ====================================================
if CFG.wandb:
    import wandb
    
    def class2dict(f):
            return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))
        
    project='Feedback Prize - English Language Learning'
    if CFG.DEBUG:
        group = "DEBUG"
    else:
        group = model_name
    wandb_name = f"{CFG.file_name}_{date2}"
    job_type = CFG.file_name  #"train"

    wandb_api = "your_id"
    wandb.login(key=wandb_api)
    anony = None
    run = wandb.init(project=project, 
                         name = wandb_name,
                         config=class2dict(CFG),
                         group=group,
                         job_type=job_type,
                         anonymous=anony)

In [4]:
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
#     os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(42)

# Load DataFrame

In [5]:
df = pd.read_csv("/home/jupyter/feedback-prize-english-language-learning/train.csv")
fold = np.load("/home/jupyter/output/fold/5folds.npy")
df['fold'] = fold

if CFG.DEBUG:
    df = df.head(5)
    df["fold"] = list(range(5))
df['fold'].value_counts()

1    783
0    782
4    782
3    782
2    782
Name: fold, dtype: int64

# Model Config

In [6]:
MAX_LENGTH = 512
BATCH_SIZE = 8
ROBERTA_MODEL = "/home/jupyter/models_tensorflow/roberta-base/"

Why we should disable dropout in regression task, check this [discussion](https://www.kaggle.com/competitions/commonlitreadabilityprize/discussion/260729).

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(ROBERTA_MODEL)
#tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')

cfg = transformers.AutoConfig.from_pretrained(ROBERTA_MODEL)
cfg.hidden_dropout_prob = 0
cfg.attention_probs_dropout_prob = 0
#cfg.save_pretrained(OUTPUT_DIR+'/tokenizer/')

# Data Process Function

To make use of HugggingFace RoBERTa model, we have to tokenize our input texts as the pretrained RoBERTa model requires.

In [8]:
def roberta_encode(texts, tokenizer=tokenizer):
    input_ids = []
    attention_mask = []
    
    for text in texts.tolist():
        token = tokenizer(text, 
                          add_special_tokens=True, 
                          max_length=MAX_LENGTH, 
                          return_attention_mask=True, 
                          return_tensors="np", 
                          truncation=True, 
                          padding='max_length')
        input_ids.append(token['input_ids'][0])
        attention_mask.append(token['attention_mask'][0])
    
    return np.array(input_ids, dtype="int32"), np.array(attention_mask, dtype="int32")

In [9]:
def get_dataset(df):
    inputs = roberta_encode(df['full_text'])
    targets = np.array(df[CFG.target_cols], dtype="float32")
    return inputs, targets

# MeanPool Layer

Instead of using '[CLS]' token, MeanPool layer averaging roberta's last hidden states along the sequence axis with masking out padding tokens.

In [10]:
class MeanPool(tf.keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        embedding_sum = tf.reduce_sum(inputs * broadcast_mask, axis=1)
        mask_sum = tf.reduce_sum(broadcast_mask, axis=1)
        mask_sum = tf.math.maximum(mask_sum, tf.constant([1e-9]))
        return embedding_sum / mask_sum

# Model

In [11]:
def get_model():
    input_ids = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), dtype=tf.int32, name="input_ids"
    )

    attention_masks = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), dtype=tf.int32, name="attention_masks"
    )

    roberta_model = transformers.TFAutoModel.from_pretrained(ROBERTA_MODEL, config=cfg)

    ## re-init#
    REINIT_LAYERS = 1
    normal_initializer = tf.keras.initializers.GlorotUniform()
    zeros_initializer = tf.keras.initializers.Zeros()
    ones_initializer = tf.keras.initializers.Ones()

    for encoder_block in roberta_model.roberta.encoder.layer[-REINIT_LAYERS:]:
        for layer in encoder_block.submodules:
            if isinstance(layer, tf.keras.layers.Dense):
                layer.kernel.assign(normal_initializer(shape=layer.kernel.shape, dtype=layer.kernel.dtype))
                if layer.bias is not None:
                    layer.bias.assign(zeros_initializer(shape=layer.bias.shape, dtype=layer.bias.dtype))

            elif isinstance(layer, tf.keras.layers.LayerNormalization):
                layer.beta.assign(zeros_initializer(shape=layer.beta.shape, dtype=layer.beta.dtype))
                layer.gamma.assign(ones_initializer(shape=layer.gamma.shape, dtype=layer.gamma.dtype))
    ####


    roberta_output = roberta_model.roberta(
        input_ids, attention_mask=attention_masks
    )
    x = roberta_output.last_hidden_state
    x = MeanPool()(x, mask=attention_masks)
    x = layers.Dense(6, activation='sigmoid')(x)
    output = layers.Rescaling(scale=4.0, offset=1.0)(x)
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)

    ### LLRD###
    layer_list = [roberta_model.roberta.embeddings] + list(roberta_model.roberta.encoder.layer)
    layer_list.reverse()

    INIT_LR = 1e-5
    LLRDR = 0.9
    LR_SCH_DECAY_STEPS = 1600 # 2 * len(train_df) // BATCH_SIZE

    lr_schedules = [tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=INIT_LR * LLRDR ** i, 
        decay_steps=LR_SCH_DECAY_STEPS, 
        decay_rate=0.3) for i in range(len(layer_list))]
    lr_schedule_head = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4, 
        decay_steps=LR_SCH_DECAY_STEPS, 
        decay_rate=0.3)

    optimizers = [tf.keras.optimizers.Adam(learning_rate=lr_sch) for lr_sch in lr_schedules]

    optimizers_and_layers = [(tf.keras.optimizers.Adam(learning_rate=lr_schedule_head), model.layers[-4:])] +\
        list(zip(optimizers, layer_list))

    optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

    model.compile(optimizer=optimizer,
                 loss='huber_loss',
                 metrics=[tf.keras.metrics.RootMeanSquaredError()],
                 )

    return model

In [12]:
tf.keras.backend.clear_session()
model = get_model()
model.summary()

2022-11-01 10:20:44.981010: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 10:20:44.983311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 10:20:44.985088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 10:20:44.987449: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 512)]        0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 124645632   input_ids[0][0]                  
                                                                 attention_masks[0][0]            
__________________________________________________________________________________________________
mean_pool (MeanPool)            (None, 768)          0           roberta[0][0]                

In [13]:
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

# 5 Folds Training Loop

In [ ]:
oof = []
valid_rmses = []
for fold in CFG.trn_fold:
    print(f'\n-----------FOLD {fold} ------------')
    
    #Create dataset
    train_df = df[df['fold'] != fold].reset_index(drop=True)
    valid_df = df[df['fold'] == fold].reset_index(drop=True)
    train_dataset = get_dataset(train_df)
    valid_dataset = get_dataset(valid_df)
    
    print('Data prepared.')
    print(f'Training data input_ids shape: {train_dataset[0][0].shape} dtype: {train_dataset[0][0].dtype}') 
    print(f'Training data attention_mask shape: {train_dataset[0][1].shape} dtype: {train_dataset[0][1].dtype}')
    print(f'Training data targets shape: {train_dataset[1].shape} dtype: {train_dataset[1].dtype}')
    print(f'Validation data input_ids shape: {valid_dataset[0][0].shape} dtype: {valid_dataset[0][0].dtype}')
    print(f'Validation data attention_mask shape: {valid_dataset[0][1].shape} dtype: {valid_dataset[0][1].dtype}')
    print(f'Validation data targets shape: {valid_dataset[1].shape} dtype: {valid_dataset[1].dtype}')
    
    #Create model
    tf.keras.backend.clear_session()
    model = get_model()
    
    callbacks = [
                tf.keras.callbacks.ModelCheckpoint(OUTPUT_DIR+f"best_model_fold{fold}.h5",
                                                   monitor="val_loss",
                                                   mode="min",
                                                   save_best_only=True,
                                                   verbose=1,
                                                   save_weights_only=True,),
                tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 min_delta=1e-5, 
                                                 patience=3, 
                                                 verbose=1,
                                                 mode='min',)
                ]
    history = model.fit(x=train_dataset[0],
                        y=train_dataset[1],
                        validation_data=valid_dataset, 
                        epochs=CFG.epochs,
                        shuffle=True,
                        batch_size=BATCH_SIZE,
                        callbacks=callbacks
                       )
      
    valid_rmses.append(np.min(history.history['val_root_mean_squared_error']))
    
    model2 = get_model()
    model2.load_weights(OUTPUT_DIR+f"best_model_fold{fold}.h5")
    pred = model2.predict(valid_dataset[0], batch_size=BATCH_SIZE)
    df.loc[df['fold'] == fold,[f"pred_{c}" for c in CFG.target_cols]] = pred
    _oof = df[df["fold"]==fold]
    oof.append(_oof)

    score, scores = get_score(_oof[CFG.target_cols].to_numpy(), pred)
    
    LOGGER.info(f"========== fold: {fold} result ==========")
    LOGGER.info(f'Score: {score:.4f}  Scores: {scores}')
    
    del train_dataset, valid_dataset, train_df, valid_df,model,model2,_oof,score,scores
    gc.collect()
    
    if CFG.wandb:
        for epoch in range(len(history.history['val_root_mean_squared_error'])):
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_score": history.history['root_mean_squared_error'][epoch], 
                       f"[fold{fold}] avg_val_score": history.history['val_root_mean_squared_error'][epoch],})
            
oof = pd.concat(oof)
oof.to_pickle(OUTPUT_DIR+f'oof_df.pkl')
score, scores = get_score(oof[CFG.target_cols].to_numpy(), oof[[f"pred_{c}" for c in CFG.target_cols]].to_numpy())
LOGGER.info(f"========== result ==========")
LOGGER.info(f'Score: {score:.4f}  Scores: {scores}')

if CFG.wandb:
        wandb.config.update(class2dict(CFG))
        wandb.finish()

In [ ]:
print(f'{len(valid_rmses)} Folds validation RMSE:\n{valid_rmses}')
print(f'Local CV Average score: {np.mean(valid_rmses)}')

In [ ]:
if CFG.TO_KAGGLE:
    UPLOAD_DIR = OUTPUT_DIR
    EX_NO = f"{CFG.model}-{CFG.file_name}" # 実験番号などを入れる、folderのpathにする
    USERID = 'your_id'


    def dataset_upload():
        import json
        from kaggle.api.kaggle_api_extended import KaggleApi

        id = f'{USERID}/{EX_NO}'

        dataset_metadata = {}
        dataset_metadata['id'] = id
        dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
        dataset_metadata['title'] = f'{EX_NO}'

        with open(UPLOAD_DIR +'dataset-metadata.json', 'w') as f:
            json.dump(dataset_metadata, f, indent=4)

        api = KaggleApi()
        api.authenticate()

        # データセットがない場合
        if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
            api.dataset_create_new(folder=UPLOAD_DIR,
                                   convert_to_csv=False,
                                   dir_mode='skip')
            
            #apiコマンドを書き込む
            f = open(f'{CFG.model}_api_command.txt', 'a')
            api_command = f"!kaggle datasets download -d hiroki8383/{EX_NO}\n"
            f.write(api_command)
            f.close()
            
            #フォルダーを削除
            if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
                remove_files = glob.glob(OUTPUT_DIR+"*")
                remove_files.remove(OUTPUT_DIR+"oof_df.pkl")
                for file in remove_files:
                    os.remove(file)
                print("folder upload")
            else:
                print("folder not upload")
            
            
        # データセットがある場合→更新されない場合がある（後で原因追及)
        else:
            print("this folder exsits")
            # api.dataset_create_version(folder=UPLOAD_DIR,
            #                            version_notes='update',
            #                            convert_to_csv=False,
            #                            delete_old_versions=False,
            #                            dir_mode='zip')

        

        
    dataset_upload()

In [ ]:
if not CFG.DEBUG:
    def to_write_score(CFG):
        df = pd.read_csv(CFG.score_path)
        def get_result2(oof_df):
                labels = oof_df[CFG.target_cols].values
                preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
                score, scores = get_score(labels, preds)
                LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
                return score,scores

        score,scores = get_result2(oof)
        name = "-".join(OUTPUT_DIR.split("/")[-4:-2])
        base = {"name":name,"score":score,"memo":CFG.MEMO} 
        base.update(dict(zip(CFG.target_cols,scores)))
        df = df.append(base,ignore_index=True)
        df.to_csv(CFG.score_path,index=False)
    to_write_score(CFG)

In [ ]:
df = pd.read_csv(CFG.score_path)
df